### Ribosome ramp

Improved Ribosome-Footprint and mRNA Measurements Provide Insights into Dynamics and Regulation of Yeast Translation

http://www.cell.com/cell-reports/abstract/S2211-1247(16)30021-3

In [15]:
import cPickle as pkl
import numpy as np
import pandas as pd
import shelve

In [6]:
filepath_general = "../results/updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20151120_2051_3600s.p"
filepath_polysomes = "../results/ribosome_timecourses_updated Shah transcriptome, full exome, no decay, updated initiation rates according to Shah_20151120_2052.db"

Regular results:

In [10]:
results = pkl.load(open(filepath_general))

In [11]:
print results.keys()

['n_tRNA', 'description', 'n_ribosomes', 'timerange', 'proteome', 'transcriptome', 'duration', 'time_stamp', 'timecourses']


Detailed results/ribosome timecourses:

In [7]:
d = shelve.open(filepath_polysomes)

Generate the ribosome lattice dict (for every gene the total of ribos at all positions):

In [13]:
ribo_lattice_dict = {}

for mRNA in results['transcriptome']:
    n_nucleotides = len(mRNA.sequence)
    last_polysome = d['mRNA_'+str(mRNA.index).zfill(5)][-1]  # we look at last polysome to catch steady state

    # Select only every third element (the others are zero anyway):
    # translate the model's polysome format to lattice of ones and zeros:
    ribos = np.array([1 if pos in last_polysome else 0 for pos in range(0, n_nucleotides, 3)])
    
    if mRNA.geneID in ribo_lattice_dict:  # if this gene has been encountered before
        ribo_lattice_dict[mRNA.geneID] += ribos
    else:                                 # if this gene is new
        ribo_lattice_dict[mRNA.geneID] = ribos

In [14]:
avg_ribos_per_gene = {}

for gene in ribo_lattice_dict:
    avg_ribos_per_gene[gene] = np.mean(ribo_lattice_dict[gene])

In [16]:
pd.Series(avg_ribos_per_gene).describe()

count    4475.000000
mean        0.120880
std         0.573657
min         0.000000
25%         0.007296
50%         0.018478
75%         0.053223
max        12.544326
dtype: float64

Check by means of an example:

In [28]:
print sum(ribo_lattice_dict['YIL140W'])*1.0/len(ribo_lattice_dict['YIL140W'])
print avg_ribos_per_gene['YIL140W']
print np.mean(ribo_lattice_dict['YIL140W'])

0.0109223300971
0.0109223300971
0.0109223300971


Define Weinberg's z (eq. S7):